<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Stockholm_Sweden_NASAPower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#pip install meteostat

In [37]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
# Stockholm, Sweden
LAT = 59.3293
LON = 18.0686

# Date Range: Full Year 2024
START_DATE = "20240101"
END_DATE = "20241231"

# API Endpoint
base_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"

# Parameters:
# T2M = Temperature (2m)
# WS10M = Wind Speed (10m)
# ALLSKY_SFC_SW_DWN = Global Horizontal Irradiance (GHI)
# ALLSKY_SFC_SW_DNI = Direct Normal Irradiance (DNI)
# ALLSKY_SFC_SW_DIFF = Diffuse Horizontal Irradiance (DHI)
params = {
    "parameters": "T2M,WS10M,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF",
    "community": "RE", # Renewable Energy
    "longitude": LON,
    "latitude": LAT,
    "start": START_DATE,
    "end": END_DATE,
    "format": "JSON"
}

print(f"Fetching NASA POWER Data for Stockholm ({START_DATE}-{END_DATE})...")
print("Request sent. Waiting for NASA server...")

try:
    r = requests.get(base_url, params=params)

    if r.status_code == 200:
        data = r.json()

        # NASA structure: properties -> parameter -> {TimeStamp: Value}
        properties = data['properties']['parameter']

        # Use T2M keys as the master timeline
        timestamps = sorted(properties['T2M'].keys())

        records = []
        for ts in timestamps:
            records.append({
                'Time_Str': ts,
                'Temperature_C': properties['T2M'].get(ts),
                'Wind_Speed_ms': properties['WS10M'].get(ts),
                'GHI_Wm2': properties['ALLSKY_SFC_SW_DWN'].get(ts),
                'DNI_Wm2': properties['ALLSKY_SFC_SW_DNI'].get(ts),
                'DHI_Wm2': properties['ALLSKY_SFC_SW_DIFF'].get(ts)
            })

        df = pd.DataFrame(records)

        # Convert Strings to DateTime
        df['Time'] = pd.to_datetime(df['Time_Str'], format='%Y%m%d%H')
        df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
        df['Hour'] = df['Time'].dt.strftime('%H:%M')

        # Clean Missing Data (-999 -> NaN)
        df = df.replace(-999, pd.NA)

        # Final Selection
        cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2', 'DNI_Wm2', 'DHI_Wm2']
        df = df[cols]

        print("-" * 30)
        print(df.head())
        print("-" * 30)

        filename = "Stockholm_NASA_POWER_2024.csv"
        df.to_csv(filename, index=False)
        print(f"Success! Data saved to {filename}")

    else:
        print(f"Error {r.status_code}: {r.text}")

except Exception as e:
    print(f"Connection failed: {e}")

Fetching NASA POWER Data for Stockholm (20240101-20241231)...
Request sent. Waiting for NASA server...
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms  GHI_Wm2  DNI_Wm2  DHI_Wm2
0  2024-01-01  00:00          -4.96           2.89      0.0      0.0      0.0
1  2024-01-01  01:00          -4.84           2.90      0.0      0.0      0.0
2  2024-01-01  02:00          -4.91           2.97      0.0      0.0      0.0
3  2024-01-01  03:00          -5.01           2.94      0.0      0.0      0.0
4  2024-01-01  04:00          -4.73           3.35      0.0      0.0      0.0
------------------------------
Success! Data saved to Stockholm_NASA_POWER_2024.csv
